In [9]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait 

# Automates downloading and setting up of Chrome Driver
from webdriver_manager.chrome import ChromeDriverManager


In [14]:
service = Service(ChromeDriverManager().install()) 

driver = webdriver.Chrome(service=service)

driver.get("https://www.scraperapi.com/web-scraping/selenium/")

